In [1]:
import estnltk
import os
import sys
import time
import torch

import pandas as pd
import numpy as np

from estnltk import Text
from estnltk.converters import text_to_json, json_to_text

from sklearn.model_selection import train_test_split

In [2]:
#read - train, test, total
#veerud - , LOC, PER, ORG, total märgendid, laused, sonad

def split_data(df):
    np.random.seed(42)
    unique_sentences = df['sentence_id'].unique()
    np.random.shuffle(unique_sentences)
    split_idx = int(len(unique_sentences) * 0.8)
    train_sentences = unique_sentences[:split_idx]
    test_sentences = unique_sentences[split_idx:]
    train_df = df[df['sentence_id'].isin(train_sentences)]
    test_df = df[df['sentence_id'].isin(test_sentences)]
    return train_df, test_df

all_df = pd.read_csv('data/klaster/filtered_tudeng_bio.csv')
train_df, test_df = split_data(all_df)
tags = ['LOC', 'PER', 'ORG']

dfs = [train_df, test_df, all_df]
names = ['train', 'test', 'all']

print(len(train_df['sentence_id'].unique()), len(test_df['sentence_id'].unique()), len(all_df['sentence_id'].unique()))

rows = []

for df, name in zip(dfs, names):
    row = [name]
    for tag in tags:
        row.append(len(df[df['labels'] == f'B-{tag}']))
    row.append(sum(row[1:4]))
    row.append(len(df['sentence_id'].unique()))
    row.append(len(df))
    rows.append(row)

result_df = pd.DataFrame(data=rows, columns=['name', 'LOC_count', 'PER_count', 'ORG_count', 'total_count', 'sentences', 'words'])
result_df

10419 2605 13024


,name,LOC_count,PER_count,ORG_count,total_count,sentences,words
0,train,3630,10323,7654,21607,10419,244032
1,test,900,2502,1931,5333,2605,59995
2,all,4530,12825,9585,26940,13024,304027


In [41]:
def split_data(df):
    np.random.seed(42)
    unique_sentences = df['sentence_id'].unique()
    np.random.shuffle(unique_sentences)
    split_idx = int(len(unique_sentences) * 0.8)
    train_sentences = unique_sentences[:split_idx]
    test_sentences = unique_sentences[split_idx:]
    train_df = df[df['sentence_id'].isin(train_sentences)]
    test_df = df[df['sentence_id'].isin(test_sentences)]
    return train_df, test_df

all_df = pd.read_csv('data/klaster/filtered_tudeng_bio.csv')
train, test = split_data(all_df)
print(train['sentence_id'].unique())
print(test['sentence_id'].unique())

[    0     2     3 ... 13020 13022 13023]
[    1     4     5 ... 13007 13010 13021]


In [33]:
import pandas as pd
import numpy as np

def split_data(df):
    np.random.seed(42)  # Setting the same seed each time
    unique_sentences = df['sentence_id'].unique()
    np.random.shuffle(unique_sentences)
    split_idx = int(len(unique_sentences) * 0.8)
    train_sentences = unique_sentences[:split_idx]  # Fixed typo from 'rain' to 'train'
    test_sentences = unique_sentences[split_idx:]
    
    # Return the sentence IDs for comparison
    return train_sentences, test_sentences

# Load your data
all_df = pd.read_csv('data/klaster/filtered_tudeng_bio.csv')

# Store the first split result to compare with others
first_train, first_test = split_data(all_df)

# Check if splits are identical across multiple runs
print("Checking consistency across 10 iterations:")
for i in range(10):
    train_sentences, test_sentences = split_data(all_df)
    
    # Check if the current split matches the first split
    train_identical = np.array_equal(train_sentences, first_train)
    test_identical = np.array_equal(test_sentences, first_test)
    
    print(f"Iteration {i+1}: Train identical: {train_identical}, Test identical: {test_identical}")
    
    # Optionally, if they're not identical, you could print the differences:
    if not train_identical:
        print(f"  First few train differences: {set(train_sentences[:5]) - set(first_train[:5])}")

Checking consistency across 10 iterations:
Iteration 1: Train identical: True, Test identical: True
Iteration 2: Train identical: True, Test identical: True
Iteration 3: Train identical: True, Test identical: True
Iteration 4: Train identical: True, Test identical: True
Iteration 5: Train identical: True, Test identical: True
Iteration 6: Train identical: True, Test identical: True
Iteration 7: Train identical: True, Test identical: True
Iteration 8: Train identical: True, Test identical: True
Iteration 9: Train identical: True, Test identical: True
Iteration 10: Train identical: True, Test identical: True


In [53]:
final_df = pd.DataFrame(columns=['attribute', 'estbertner', 'tudeng', 'opetaja'])
models = ['estbertner', 'tudeng', 'opetaja']

#rows - 5 koikumist, nende keskimine?, kuhugi precision ja callback

for filename in os.listdir('final_results/pre_clean_fixed/'):
    results_df = pd.read_csv(f'final_results/pre_clean_fixed/{filename}')
    if len(filename.split('_')) == 3:
        short_name = filename.split('_')[1]
    else:
        short_name = '_'.join(filename.split('_')[1:3])
    temp_row = [short_name]
    for model in models:
        value = results_df[(results_df['model'] == model) & (results_df['tag'] == 'Overall')]['f1'].iloc[0]
        temp_row.append(value)

    final_df.loc[len(final_df)] = temp_row
final_df.loc[len(final_df)] = final_df.mean(numeric_only=True)

final_df

,attribute,estbertner,tudeng,opetaja
0,1922-04-24_1936-09-07,0.6025,0.71280,0.53220
1,1927-03-28_1941-01-03,0.5600,0.63410,0.47230
2,1932-01-25,0.4025,0.66380,0.36180
3,1934-10-15,0.5899,0.73730,0.47000
4,1935-09-30,0.6876,0.76730,0.48190
5,NaN,0.5685,0.70306,0.46364


In [14]:
best_models = {}

input_folder = 'final_results/pre_clean_fixed'

models_in_all_steps = set()

for filename in os.listdir(input_folder):
    df = pd.read_csv(os.path.join(input_folder, filename))
    
    if len(models_in_all_steps) == 0:
        models_in_all_steps.update(df['model'].unique())
    else:
        models_in_all_steps = models_in_all_steps & set(df['model'].unique())
 
models_in_all_steps.difference_update({'estbertner', 'opetaja', 'tudeng'})
print(models_in_all_steps)
print(len(models_in_all_steps))

{'model26000_65', 'model13000_70', 'model19500_60', 'model13000_65', 'model19500_65', 'model32500_65', 'model6500_65', 'model19500_70', 'model32500_60', 'model26000_60', 'model6500_75', 'model13000_60', 'model6500_60', 'model39000_60', 'model6500_70'}
15


In [33]:
input_folder = 'final_results/pre_clean_fixed'
all_dfs = []

models_in_all_steps = set()

for filename in os.listdir(input_folder):
    df = pd.read_csv(os.path.join(input_folder, filename))
    df['filename'] = filename
    all_dfs.append(df)
    if len(models_in_all_steps) == 0:
        models_in_all_steps.update(df['model'].unique())
    else:
        models_in_all_steps = models_in_all_steps & set(df['model'].unique())

models_in_all_steps.difference_update({'estbertner', 'opetaja', 'tudeng'})
print(models_in_all_steps)

combined_df = pd.concat(all_dfs, ignore_index=True)
combined_df['lavend'] = combined_df['model'].str.split('_').str[1]

print('-'*10)
avg_f1_by_lavend = combined_df[combined_df['tag'] == 'Overall'].groupby(['lavend'])['f1'].mean().sort_index()
print(avg_f1_by_lavend)
print('-'*10)

best_lavend = avg_f1_by_lavend.idxmax()
print(best_lavend)

best_lavend_models = []
for model_name in models_in_all_steps:
    if model_name.split('_')[1] == best_lavend:
        best_lavend_models.append(model_name)
print(best_lavend_models)

best_models_from_best_lavend = combined_df[(combined_df['tag'] == 'Overall') & (combined_df['model'].isin(best_lavend_models))]
#best_models_from_best_lavend

f1_stats_by_model_lavend = best_models_from_best_lavend.groupby(['model', 'lavend'])['f1'].agg(['mean', 'std']).sort_index()

print(f1_stats_by_model_lavend)


{'model19500_70', 'model6500_70', 'model32500_60', 'model13000_70', 'model13000_65', 'model19500_65', 'model19500_60', 'model6500_60', 'model6500_65', 'model13000_60', 'model26000_60', 'model6500_75', 'model32500_65', 'model39000_60', 'model26000_65'}
----------
lavend
60    0.688787
65    0.679343
70    0.660819
75    0.649409
80    0.603817
85    0.435280
90    0.224080
95    0.051340
Name: f1, dtype: float64
----------
60
['model32500_60', 'model19500_60', 'model6500_60', 'model13000_60', 'model26000_60', 'model39000_60']
                         mean       std
model         lavend                   
model13000_60 60      0.68402  0.075221
model19500_60 60      0.68474  0.070696
model26000_60 60      0.68714  0.072834
model32500_60 60      0.69464  0.081142
model39000_60 60      0.69240  0.078295
model6500_60  60      0.65516  0.066040


In [4]:
#combined_df[(combined_df['tag'] == 'Overall')].groupby(['model', 'lavend'])['f1'].agg(['mean', 'std']).sort_index()

In [64]:
input_folder = 'final_results/pre_clean_fixed'
all_dfs = []

models_in_all_steps = set()

for filename in os.listdir(input_folder):
    df = pd.read_csv(os.path.join(input_folder, filename))
    df['filename'] = filename
    all_dfs.append(df)
    if len(models_in_all_steps) == 0:
        models_in_all_steps.update(df['model'].unique())
    else:
        models_in_all_steps = models_in_all_steps & set(df['model'].unique())

models_in_all_steps.difference_update({'estbertner', 'opetaja', 'tudeng'})
print(models_in_all_steps)

filter_models = ['tudeng', 'estbertner', 'opetaja']

combined_df = pd.concat(all_dfs, ignore_index=True)
combined_df['lavend'] = combined_df['model'].str.split('_').str[1]

combined_df = combined_df[(~combined_df['model'].isin(filter_models)) & (combined_df['tag'] == 'Overall')]
group_df = combined_df.groupby('filename')

for lavend in grouped_df['lavend'].unique():
    print(lavend)
    for filename, grouped_df in group_df:
        temp_df = grouped_df[grouped_df['lavend'] == lavend].copy()
        temp_df['num_tags'] = temp_df['model'].apply(lambda x: x.split('_')[0][5:])
        temp_df['num_tags'] = temp_df['num_tags'].astype(int)
        temp_df = temp_df.sort_values(by='num_tags', ascending=True)
        #print(temp_df)
        last_f1 = 0
        best_name = ''
        for index, row in temp_df.iterrows():
            if float(row['f1']) > last_f1:
                last_f1 = float(row['f1'])
                best_name = row['model']
            else:
                break
        print(filename, lavend, best_name, last_f1)

{'model26000_60', 'model19500_65', 'model6500_60', 'model32500_65', 'model13000_70', 'model19500_60', 'model39000_60', 'model32500_60', 'model13000_60', 'model19500_70', 'model6500_75', 'model26000_65', 'model13000_65', 'model6500_70', 'model6500_65'}
80
results_1922-04-24_1936-09-07_fixed.csv 80 model6500_80 0.6182
results_1927-03-28_1941-01-03_fixed.csv 80 model5394_80 0.5806
results_1932-01-25_fixed.csv 80 model6500_80 0.5
results_1934-10-15_fixed.csv 80 model5115_80 0.5976
results_1935-09-30_fixed.csv 80 model6500_80 0.6707
95
results_1922-04-24_1936-09-07_fixed.csv 95 model103_95 0.0692
results_1927-03-28_1941-01-03_fixed.csv 95  0
results_1932-01-25_fixed.csv 95 model181_95 0.1875
results_1934-10-15_fixed.csv 95  0
results_1935-09-30_fixed.csv 95  0
90
results_1922-04-24_1936-09-07_fixed.csv 90 model759_90 0.1272
results_1927-03-28_1941-01-03_fixed.csv 90 model344_90 0.2831
results_1932-01-25_fixed.csv 90 model1085_90 0.2723
results_1934-10-15_fixed.csv 90 model443_90 0.0032
resu

In [9]:
input_folder = 'final_results/pre_clean_fixed'
all_dfs = []

models_in_all_steps = set()

for filename in os.listdir(input_folder):
    df = pd.read_csv(os.path.join(input_folder, filename))
    df['filename'] = filename
    all_dfs.append(df)
    if len(models_in_all_steps) == 0:
        models_in_all_steps.update(df['model'].unique())
    else:
        models_in_all_steps = models_in_all_steps & set(df['model'].unique())

models_in_all_steps.difference_update({'estbertner', 'opetaja', 'tudeng'})
print(models_in_all_steps)

filter_models = ['tudeng', 'estbertner', 'opetaja']

combined_df = pd.concat(all_dfs, ignore_index=True)
combined_df['lavend'] = combined_df['model'].str.split('_').str[1]

combined_df = combined_df[(~combined_df['model'].isin(filter_models)) & (combined_df['tag'] == 'Overall')]
group_df = combined_df.groupby('filename')

sorted_lavends = sorted(combined_df['lavend'].unique())

for lavend in sorted_lavends:
    #print(lavend)
    f1s = []
    for filename, grouped_df in group_df:
        temp_df = grouped_df[grouped_df['lavend'] == lavend].copy()
        temp_df['num_tags'] = temp_df['model'].apply(lambda x: x.split('_')[0][5:])
        temp_df['num_tags'] = temp_df['num_tags'].astype(int)
        temp_df = temp_df.sort_values(by='num_tags', ascending=True)
        #print(temp_df)
        last_f1 = 0
        best_name = ''
        for index, row in temp_df.iterrows():
            if float(row['f1']) > last_f1:
                last_f1 = float(row['f1'])
                best_name = row['model']
            else:
                break
        f1s.append(last_f1)
        #print(filename, lavend, best_name, last_f1)
    #print(lavend, round(sum(f1s)/len(f1s), 4), f1s)
    print(lavend, round(sum(f1s)/len(f1s), 4))

{'model6500_65', 'model6500_70', 'model6500_75', 'model6500_60', 'model19500_60', 'model32500_65', 'model19500_65', 'model39000_60', 'model13000_70', 'model13000_65', 'model26000_60', 'model26000_65', 'model19500_70', 'model32500_60', 'model13000_60'}
60 0.6887
65 0.6926
70 0.6822
75 0.657
80 0.5934
85 0.4353
90 0.2241
95 0.0513


In [45]:
input_folder = 'final_results/pre_clean_fixed_new'
all_dfs = []

models_in_all_steps = set()

for filename in os.listdir(input_folder):
    df = pd.read_csv(os.path.join(input_folder, filename))
    df['filename'] = filename
    all_dfs.append(df)
    if len(models_in_all_steps) == 0:
        models_in_all_steps.update(df['model'].unique())
    else:
        models_in_all_steps = models_in_all_steps & set(df['model'].unique())

models_in_all_steps.difference_update({'estbertner', 'opetaja', 'tudeng'})
print(models_in_all_steps)

combined_df = pd.concat(all_dfs, ignore_index=True)
combined_df['lavend'] = combined_df['model'].str.split('_').str[1]

best_lavend = '65'
print(best_lavend)

best_lavend_models = ['estbertner', 'opetaja']
for model_name in models_in_all_steps:
    if model_name.split('_')[1] == best_lavend:
        best_lavend_models.append(model_name)
print(best_lavend_models)

best_models_from_best_lavend = combined_df[(combined_df['tag'] == 'Overall') & (combined_df['model'].isin(['estbertner', 'opetaja']))]
#best_models_from_best_lavend
print(len(best_models_from_best_lavend))
f1_stats_by_model_lavend = best_models_from_best_lavend.groupby(['model'])['f1'].agg(['mean', 'std']).sort_index()

print(f1_stats_by_model_lavend)

{'model6500_65', 'model6500_70', 'model6500_75', 'model6500_60', 'model19500_60', 'model32500_65', 'model19500_65', 'model39000_60', 'model13000_70', 'model13000_65', 'model26000_60', 'model26000_65', 'model19500_70', 'model32500_60', 'model13000_60'}
65
['estbertner', 'opetaja', 'model6500_65', 'model32500_65', 'model19500_65', 'model13000_65', 'model26000_65']
10
               mean       std
model                        
estbertner  0.56850  0.104196
opetaja     0.46364  0.062291


In [25]:
models = ['model32500_65', 'opetaja', 'estbertner', 'tudeng']
input_folder = 'final_results/pre_clean_fixed_new'
all_dfs = []

for filename in os.listdir(input_folder):
    df = pd.read_csv(os.path.join(input_folder, filename))
    df['filename'] = filename
    all_dfs.append(df)

combined_df = pd.concat(all_dfs, ignore_index=True)
combined_df['lavend'] = combined_df['model'].str.split('_').str[1]

stats = ['precision', 'recall', 'f1']

specific_models_df = combined_df[(combined_df['tag'] == 'Overall') & (combined_df['model'].isin(models))]
for stat in stats:
    results = specific_models_df.groupby(['model'])[stat].agg(['mean', 'std']).sort_index()
    print(stat)
    print(results)
    print()

precision
                  mean       std
model                           
estbertner     0.66546  0.147635
model32500_65  0.69714  0.105477
opetaja        0.42124  0.083282
tudeng         0.68262  0.089578

recall
                  mean       std
model                           
estbertner     0.50000  0.084498
model32500_65  0.68068  0.043715
opetaja        0.52286  0.041060
tudeng         0.70400  0.044707

f1
                  mean       std
model                           
estbertner     0.56850  0.104196
model32500_65  0.68738  0.073589
opetaja        0.46364  0.062291
tudeng         0.69166  0.063019



In [31]:
models = ['estbertner', 'opetaja', 'model32500_65', 'tudeng']
input_folder = 'final_results/pre_clean_fixed_new'
all_dfs = []

for filename in os.listdir(input_folder):
    df = pd.read_csv(os.path.join(input_folder, filename))
    df['filename'] = filename
    all_dfs.append(df)

combined_df = pd.concat(all_dfs, ignore_index=True)
combined_df['lavend'] = combined_df['model'].str.split('_').str[1]

# Filter for specific models and the Overall tag
specific_models_df = combined_df[(combined_df['tag'] == 'Overall') & 
                                (combined_df['model'].isin(models))]

# Create an empty DataFrame to store results
results_df = pd.DataFrame(index=['precision', 'recall', 'f1'])

# Populate the DataFrame
for model in models:
    model_data = specific_models_df[specific_models_df['model'] == model]
    for stat in ['precision', 'recall', 'f1']:
        mean_val = model_data[stat].mean()
        std_val = model_data[stat].std()
        # Format as "mean ± std"
        results_df.loc[stat, model] = f"& {mean_val:.4f}\pm{std_val:.4f}"

# Print the reshaped results
print(results_df)

                  estbertner            opetaja      model32500_65  \
precision  & 0.6655\pm0.1476  & 0.4212\pm0.0833  & 0.6971\pm0.1055   
recall     & 0.5000\pm0.0845  & 0.5229\pm0.0411  & 0.6807\pm0.0437   
f1         & 0.5685\pm0.1042  & 0.4636\pm0.0623  & 0.6874\pm0.0736   

                      tudeng  
precision  & 0.6826\pm0.0896  
recall     & 0.7040\pm0.0447  
f1         & 0.6917\pm0.0630  


<>:27: SyntaxWarning: invalid escape sequence '\p'
<>:27: SyntaxWarning: invalid escape sequence '\p'
C:\Users\Norman\AppData\Local\Temp\ipykernel_9320\4053160571.py:27: SyntaxWarning: invalid escape sequence '\p'
  results_df.loc[stat, model] = f"& {mean_val:.4f}\pm{std_val:.4f}"
